In [6]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
from scipy.sparse import csr_matrix, save_npz
import os
import matplotlib.pyplot as plt


def create_sparse_matrix(data):
    '''
       This function takes a dataframe and creates a sparse user-item matrix and 
       sparse item-user matrix using csr matrix
       
       returns:
       sparse item-user matrix, user-item matrix
    '''
    # unique users
    users = list(np.sort(data.user_id.unique()))  
    
    #unique restaurants
    restuarant = list(np.sort(data_matrix.business_id.unique()))  
    
    #stars
    stars = list(data.stars) 
    
    print(users)
    print(restaurant)
    print(stars)
    #rows = data_matrix.user_id.astype(CategoricalDtype(
                                      #categories=users)).cat.codes
    #cols = data_matrix.business_id.astype(CategoricalDtype(
                                      #categories=restuarant)).cat.codes
    
    
    #sparse_restuarant_user = csr_matrix((rating, (cols, rows)),
                                  #shape=(len(restuarant), len(users)))
    #sparse_user_restuarant = csr_matrix((rating, (rows, cols)),
                                  #shape=(len(users), len(restuarant)))
    #print("Sparse matrices created : sparse_restuarant_user ",
          #sparse_restuarant_user.shape, "sparse_user_restuarant",
          #sparse_user_restuarant.shape)
    #return sparse_restuarant_user, sparse_user_restuarant

def main():
    data = pd.read_pickle("./output/review.pkl")
    create_sparse_matrix(data)

if __name__ == "__main__":
    main()

AttributeError: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/Users/poornimamuthukumar/miniconda3/lib/python3.9/site-packages/pandas/_libs/internals.cpython-39-darwin.so'>

In [4]:
restaurant = pd.read_csv('../data/clean/restaurant.csv')

In [5]:
review = pd.read_csv('../data/clean/review.csv')

In [8]:
restaurant.count()

business_id    5857
name           5857
dtype: int64

In [9]:
review.count()

business_id    687571
name           687571
user_id        687571
stars          687571
dtype: int64

In [ ]:
import os
directory = './output'
if not os.path.exists(directory):
    os.makedirs(directory)
restaurant.to_pickle("./output/restaurant.pkl")
review.to_pickle("./output/review.pkl")